# American Ninja Warrior Webscraping, Visualization, and Analysis
### Notebook 1: Data gathering and preparation

We'll use pandas for webscraping. Originally I scraped the data with the requests and Beautiful Soup libraries. However, the pandas *read_html* function makes scraping table data more convenient.

By: Jeff Hale

Imports

In [141]:
import time
import pandas as pd
import numpy as np
import plotly.express as px


The format of the url for season 10 is different than for the other seasons, so we'll start building our DataFrame by scraping season 10's data.

In [181]:
list_df_10 = pd.read_html("https://www.anwfantasy.com/runs10")
list_df_10

[                 Name  Fantasy Points                    CQ OBS    Time  \
 0          Sean Bryan            49.5             Complete (LA)  236.84   
 1       Drew Drechsel            44.0            Complete (MIA)  144.74   
 2        Josh Salinas            39.0            Complete (DAL)   141.3   
 3    Chris Wilczewski            39.0            Complete (PHL)  121.53   
 4    Najee Richardson            39.0            Complete (PHL)  173.86   
 ..                ...             ...                       ...     ...   
 297      Chris Howard             0.0      Floating Steps (IND)     NaN   
 298  Anthony DeFranco             0.0      Floating Steps (PHL)     NaN   
 299   Enedina Stanger             0.0       Floating Steps (MN)     NaN   
 300      Mikayla Fong             0.0  [1] Shrinking Steps (LA)     NaN   
 301     Milton Nkunku             0.0  [1] Shrinking Steps (LA)     NaN   
 
                  CF OBS     Time.1   Stage 1  Time.2        Stage 2  \
 0         Com

*read_html* returns a list of DataFrames, so let's grab the DataFrame and add a column for the year so that we can track that.

In [166]:
df = list_df_10[0]

df['Year'] = 10
df.head(2)

,Name,Fantasy Points,CQ OBS,Time,CF OBS,Time.1,Stage 1,Time.2,Stage 2,Time.3,Stage 3,Stage 4,Time.4,Year
0,Sean Bryan,49.5,Complete (LA),236.84,Complete (LA),369.99,Complete,126.05,Complete,248.3,Ultimate Crazy Cliffhanger,NaN,NaN,10
1,Drew Drechsel,44.0,Complete (MIA),144.74,Stair Hopper (MIA),Qualified,Complete,96.2,Complete,231.35,Ultimate Crazy Cliffhanger,NaN,NaN,10


Now we'll scrape and concatenate the data for years four to nine, which have the same columns. Years one through three and eleven have some differen columns.

In [167]:
for season in range(9, 3, -1):
    try:
        df_to_add = pd.read_html(f"https://www.anwfantasy.com/runs-{season}")[0]
        df_to_add['Year'] = season
        df = pd.concat(objs=[df, df_to_add])
        print(f"After season {season} shape is {df.shape}")
    except Exception:
        print(f"Scraping season {season} failed")


After season 9 shape is (621, 14)
After season 8 shape is (856, 14)
After season 7 shape is (1164, 14)
After season 6 shape is (1472, 14)
After season 5 shape is (1674, 14)
After season 4 shape is (1944, 14)


In [168]:
df.head(2)

,Name,Fantasy Points,CQ OBS,Time,CF OBS,Time.1,Stage 1,Time.2,Stage 2,Time.3,Stage 3,Stage 4,Time.4,Year
0,Sean Bryan,49.5,Complete (LA),236.84,Complete (LA),369.99,Complete,126.05,Complete,248.3,Ultimate Crazy Cliffhanger,NaN,NaN,10
1,Drew Drechsel,44.0,Complete (MIA),144.74,Stair Hopper (MIA),Qualified,Complete,96.2,Complete,231.35,Ultimate Crazy Cliffhanger,NaN,NaN,10


In [169]:
df.tail(2)


,Name,Fantasy Points,CQ OBS,Time,CF OBS,Time.1,Stage 1,Time.2,Stage 2,Time.3,Stage 3,Stage 4,Time.4,Year
268,Brian Lara,1.0,Log Grip (se),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
269,Beth Williams,1.0,Log Grip (sw),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4


Let's add Seasons 1-3. Those seasons have a *Boot Camp* column before *Stage 1*.

In [170]:
df.insert(loc=6, column="Boot Camp", value=np.NaN)
df.head(2)


,Name,Fantasy Points,CQ OBS,Time,CF OBS,Time.1,Boot Camp,Stage 1,Time.2,Stage 2,Time.3,Stage 3,Stage 4,Time.4,Year
0,Sean Bryan,49.5,Complete (LA),236.84,Complete (LA),369.99,NaN,Complete,126.05,Complete,248.3,Ultimate Crazy Cliffhanger,NaN,NaN,10
1,Drew Drechsel,44.0,Complete (MIA),144.74,Stair Hopper (MIA),Qualified,NaN,Complete,96.2,Complete,231.35,Ultimate Crazy Cliffhanger,NaN,NaN,10


Let's make a function to scrape and concatenate to keep our code DRY.

In [171]:
def scrape_concat(
    df_existing: pd.DataFrame, start: int, stop: int, step: int
) -> pd.DataFrame:
    """
    Scrape some number of Ninja Warrior data and concatenate it to an existing DataFrame

    Args:
        df_existing: An existing pandas DataFrame
        start: starting value for year to scrape
        stop: stopping value for year to scrape (not inclusive)
        step: step size for year to scrape

    Returns:
        df_existing: the concatenated DataFrame

    """

    for season in range(start, stop, step):
        try:
            df_to_add = pd.read_html(f"https://www.anwfantasy.com/runs-{season}")[0]
            df_to_add["Year"] = season
            df_existing = pd.concat(objs=[df_existing, df_to_add])
            print(f"After season {season} shape is {df_existing.shape}")
        except Exception:
            print(f"Scraping season {season} failed")
    return df_existing


In [172]:
df = scrape_concat(df, 3, 0, -1)
df.tail(2)

After season 3 shape is (2012, 15)
After season 2 shape is (2084, 15)
After season 1 shape is (2135, 15)


,Name,Fantasy Points,CQ OBS,Time,CF OBS,Time.1,Boot Camp,Stage 1,Time.2,Stage 2,Time.3,Stage 3,Stage 4,Time.4,Year
49,Dylan Wight,1.0,Rope Swing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
50,David Moss,1.0,Rope Swing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


Let's add season 11. Season 11 includes a new *run* number column before the other columns.

In [31]:
df.insert(loc=0, column="run", value=np.NaN)
df.head(2)


,run,Name,Fantasy Points,CQ OBS,Time,CF OBS,Time.1,Boot Camp,Stage 1,Time.2,Stage 2,Time.3,Stage 3,Stage 4,Time.4
0,NaN,Sean Bryan,49.5,Complete (LA),236.84,Complete (LA),369.99,NaN,Complete,126.05,Complete,248.3,Ultimate Crazy Cliffhanger,NaN,NaN
1,NaN,Drew Drechsel,44.0,Complete (MIA),144.74,Stair Hopper (MIA),Qualified,NaN,Complete,96.2,Complete,231.35,Ultimate Crazy Cliffhanger,NaN,NaN


In [175]:
df = scrape_concat(df, 11, 12, 1)
df.tail(2)

After season 11 shape is (2437, 16)


,Name,Fantasy Points,CQ OBS,Time,CF OBS,Time.1,Boot Camp,Stage 1,Time.2,Stage 2,Time.3,Stage 3,Stage 4,Time.4,Year,run
300,Mikayla Fong,0.0,[1] Shrinking Steps (LA),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11,39.0
301,Milton Nkunku,0.0,[1] Shrinking Steps (LA),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11,12.0


In [176]:
df.shape


(2437, 16)

Let's reset the index

In [177]:
df.reset_index(inplace=True)
df.tail(2)

,index,Name,Fantasy Points,CQ OBS,Time,CF OBS,Time.1,Boot Camp,Stage 1,Time.2,Stage 2,Time.3,Stage 3,Stage 4,Time.4,Year,run
2435,300,Mikayla Fong,0.0,[1] Shrinking Steps (LA),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11,39.0
2436,301,Milton Nkunku,0.0,[1] Shrinking Steps (LA),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11,12.0


Looks good. Let's save the DataFrame out to a file with current time info.

In [179]:
time_now = time.strftime("%Y-%m-%d-%H:%M")
df.to_csv(f"data/anw_data-{time_now}.csv")


Read the file in if needed.

In [42]:
# df = pd.read_csv(f"data/anw_data-SOMETIME.csv", index_col=0)


## Data cleaning

We need to convert any numeric columns we want to anlayze into numeric data types.

In [180]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2437 entries, 0 to 2436
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   index           2437 non-null   int64  
 1   Name            2404 non-null   object 
 2   Fantasy Points  2404 non-null   float64
 3   CQ OBS          2397 non-null   object 
 4   Time            1519 non-null   object 
 5   CF OBS          1397 non-null   object 
 6   Time.1          768 non-null    object 
 7   Boot Camp       53 non-null     object 
 8   Stage 1         662 non-null    object 
 9   Time.2          243 non-null    object 
 10  Stage 2         234 non-null    object 
 11  Time.3          56 non-null     object 
 12  Stage 3         55 non-null     object 
 13  Stage 4         4 non-null      object 
 14  Time.4          3 non-null      float64
 15  Year            2437 non-null   int64  
 16  run             302 non-null    float64
dtypes: float64(3), int64(2), object(1

In [ ]:
df["Fantasy Points"] = pd.to_numeric(df["Fantasy Points"])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2437 entries, 0 to 301
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   run             302 non-null    float64
 1   Name            2404 non-null   object 
 2   Fantasy Points  2404 non-null   float64
 3   CQ OBS          2397 non-null   object 
 4   Time            1519 non-null   object 
 5   CF OBS          1397 non-null   object 
 6   Time.1          768 non-null    object 
 7   Boot Camp       53 non-null     object 
 8   Stage 1         662 non-null    object 
 9   Time.2          243 non-null    object 
 10  Stage 2         234 non-null    object 
 11  Time.3          56 non-null     object 
 12  Stage 3         55 non-null     object 
 13  Stage 4         4 non-null      object 
 14  Time.4          3 non-null      float64
dtypes: float64(3), object(12)
memory usage: 304.6+ KB


## Analysis

Let's see what we have time to see. 👀

In [64]:
df.head(2)


,run,Name,Fantasy Points,CQ OBS,Time,CF OBS,Time.1,Boot Camp,Stage 1,Time.2,Stage 2,Time.3,Stage 3,Stage 4,Time.4
0,NaN,Sean Bryan,49.5,Complete (LA),236.84,Complete (LA),369.99,NaN,Complete,126.05,Complete,248.3,Ultimate Crazy Cliffhanger,NaN,NaN
1,NaN,Drew Drechsel,44.0,Complete (MIA),144.74,Stair Hopper (MIA),Qualified,NaN,Complete,96.2,Complete,231.35,Ultimate Crazy Cliffhanger,NaN,NaN


In [148]:
df_appearances = df["Name"].value_counts().to_frame().reset_index()
df_appearances.columns=['Name', 'Appearances']
df_appearances

,Name,Appearances
0,Ryan Stratis,11
1,David Campbell,11
2,Travis Rosen,10
3,Lorin Ball,9
4,Chris Wilczewski,9
...,...,...
1516,Omar Payton,1
1517,Zac Eddington,1
1518,Spenser Mestel,1
1519,Robert Taylor,1


In [154]:
fig = px.histogram(df_appearances, x='Appearances')
fig.update_layout(
    title_text='Most competitors make a single appearance',
    xaxis_title_text='Number of years a competitor appeared',
    yaxis_title_text='Count',
    bargap=0.1,
    showlegend=False,
)


#### How many people appeared at least 7 times in the five years?

In [158]:
df_appearances[df_appearances['Appearances'] >= 7].count()['Appearances']

22

#### Who are the overall fantasy point leaders?

In [129]:
df_totals = (
    df.groupby(["Name"])
    .sum()
    .sort_values(by="Fantasy Points", ascending=False)[["Fantasy Points"]]
)
df_totals


,Fantasy Points
Name,
Drew Drechsel,353.0
Joe Moravsky,299.5
Ryan Stratis,289.5
David Campbell,289.0
Travis Rosen,287.0
...,...
Dennis Ruelas,0.0
Charlie Escue,0.0
Tony Geronimo,0.0


In [131]:
fig2 = px.histogram(df_totals, title='Distribution of yearly fantasy points')
fig2.update_layout(
    bargap=0.1,
    showlegend=False,
    xaxis_title_text='Fantasy Points',
    yaxis_title_text='Count',
)


What are those fantasy points anyway? According to the [website](http://www.anwfantasy.com/how-to-play/), points are scored as follows.

    1 point for every obstacle cleared!

    2 points for City Qualifier course clear
    4 points for City Finals course clears
    4 points for Midoriyama/Las Vegas Stage 1 clears
    6 points for Midoriyama/Las Vegas Stage 2 clears
    8 points for Midoriyama/Las Vegas Stage 3 clears
    10 points for Total Victory
    Additonally a 0.5 point bonus is awarded to the fastest runner for each timed stage

    For example Geoff Britten's "Perfect Season" is scored:
    • 2pts for City Qualifier cleared + 6 obstacles cleared +
    • 4pts for City Finals cleared + 10 obstacles cleared (+0.5 fastest bonus) +
    • 4pts for Stage 1 cleared + 8 obstacles cleared +
    • 6pts for Stage 2 cleared + 6 obstacles cleared (+0.5 fastest bonus) +
    • 8pts for Stage 3 cleared + 8 obstacles cleared +
    • 10pts for Stage 4 cleared + 1 obstacle cleared =
    74 total points!

#### Let's see who had the most fantasy points in a given year.

In [ ]:
fig2 = px.histogram(df_totals, title='Distribution of yearly fantasy points')
fig2.update_layout(
    bargap=0.1,
    showlegend=False,
    xaxis_title_text='Fantasy Points',
    yaxis_title_text='Count',
)


#### Let's plot the top 10 cumulative fantasy point leaders.

In [137]:
df.max()

/var/folders/jt/19kddqbx06s0kvc4vjhc94qc0000gn/T/ipykernel_10253/1151452817.py:1: FutureWarning:

Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.



run               302.00
Fantasy Points     71.50
Time.4             29.65
dtype: float64

In [134]:
df[df["Fantasy Points"].max(10)]


ValueError: No axis named 10 for object type Series

In [123]:
fig3 = px.bar(
    df_fantasy.head(10),
    x="Name",
    y="Fantasy Points",
    color="Name",
    title="Fantasy point leaders (years 1-11)"
)
fig3.update_layout(
    showlegend=False,
)
